## Define the network

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Réseaux de neurones convolutifs (CNN)
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        #Une couche convolutive avec un canal d'entrée (valeur de gris), 6 canaux de sorties et un noyaux 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        #Une couche convolutive avec 6 canaux car la précédente couche possède 6 canaux de sorties
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b

        # 1ère couche connectée de taille (400x120),
        # le 16 représente le nombre de canaux de sortie dans la dernière couche convolutive
        # 5*5 la taille de l'image
        # et 120 le nombre de canaux de sortie de cette couche
        self.fc1 = nn.Linear(16 * 5 * 5, 120)

        # 120 représente le nombre de canaux de sortie de la précédente couche connectée
        # et 84 le nombre de canaux de sortie de cette couche
        self.fc2 = nn.Linear(120, 84)

        # 84 représente le nombre de canaux de sortie de la précédente couche connectée
        # et 10 le nombre de canaux de sortie de cette couche et du modèle, ici le modèle prédira 10 classes différentes
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        """
        Méthode de propagation avant, c'est à dire faire passer les données d'entrée
        dans chaques couches du réseau de neurones pour obtenir une prédiction.
        :param input: données d'entrée du réseau de neurones
        :return: données de sortie du réseau de neurones
        """

        c1 = F.relu(self.conv1(input))
        # ReLU introduit de la non-linéarité au model en remplaçant les valeurs négatives par zéro.
        s2 = F.max_pool2d(c1, (2, 2))
        # Max_pool2d divise la carte de caractéristique en fenêtre 2x2 en prenant dans chauqe fenêtre la valeur maximale.
        c3 = F.relu(self.conv2(s2))
        s4 = F.max_pool2d(c3, 2)
        s4 = torch.flatten(s4, 1)
        # On transforme une matrice multidimensionnelle en vecteur unidimensionnel
        f5 = F.relu(self.fc1(s4))
        f6 = F.relu(self.fc2(f5))
        output = self.fc3(f6)
        return output


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [5]:
# On crée une niveau avec un canal (niveaux de gris) de taille 32x32
# de valeur aléatoire de loi normale centrée réduite
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1280,  0.0220, -0.0232,  0.0365,  0.0465,  0.1030,  0.1095, -0.0656,
          0.0241,  0.0664]], grad_fn=<AddmmBackward0>)


In [6]:
net.zero_grad() #On remet à zero les gradient accumulés des itérations précédentes.

#Ici on utilisent un tenseur de variable alétoire normale centrée réduite
#Normalement on devrait utiliser en situation réel le gradient de la fonction de perte
out.backward(torch.randn(1, 10))

## Loss Function

In [7]:
output = net(input)
target = torch.randn(10)  # On établit des cible aléatoire pour tester
target = target.view(1, -1)  # On modifie la forme de target pour qu'elle soit compatible avec output
criterion = nn.MSELoss() #On définit la fonction de perte, ici MSELoss avec la bibliothèque nn

loss = criterion(output, target) # On calcul la perte en comparant output avec target
print(loss)

tensor(0.3721, grad_fn=<MseLossBackward0>)


In [9]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU
#input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> flatten
#      -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss

## Backprop

In [ ]:
net.zero_grad() #On réinitialise les gradients de tout les paramètres à zéro

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad) #On affiche les gradients des biais. Nulle ici car on appelle net.zero_grad() avant

loss.backward() #On calcule les gradients des paramètres

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad) #On affiche les gradients des biais.

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0033,  0.0033, -0.0059,  0.0049,  0.0111,  0.0020])


## Update the weights

weight = weight - learning_rate * gradient

In [10]:
learning_rate = 0.01 #Taux d'apprentissage
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate) # On met à jour les paramètres

In [11]:
import torch.optim as optim

# On crée un optimiseur de descente de gradient stochastique (SGD)
# qui met à jour les paramètre du réseau avec un taux d'appretissage à 0.01
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()  #On réinitialise les gradients de tout les paramètres à zéro
output = net(input) # On calcul l'input dans le réseau de neurones
loss = criterion(output, target) # On calcul la perte en comparant output avec target
loss.backward() #Calcul du gradient de la fonction de perte par rapport aux paramètres
optimizer.step() # On met à jour les paramètres en fonction des gradients calculés et du taux d'apprentissage